In [1]:
import json
import pandas as pd

In [2]:
from htmd.ui import * 
config(viewer='webgl')

/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:29: UserWarning: As of HTMD 1.16 the default ACEMD version for all protocols has changed to version 3. If you want to use version 2 protocols change the _version argument in the protocols or add `config(acemdversion=2)` to the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:33: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.23.2 python[3.7,<3.8.0a0,3.6,<3.7.0a0]) is available. You are currently on (1.19).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.23.2 htmd=1.23.2 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



In [3]:
from create_csv import *

In [4]:
def create_reps_all(gpcr,var_info):
    
    name=var_info[gpcr]["name"]
    print("%s (%s)" % (name,gpcr))
    pdb=var_info[gpcr]["pdb"]
    is_model=var_info[gpcr]["is_model"]
    gpcr_vars=[]
    if len(list(var_info[gpcr].keys())) >1:
        gpcr_vars=var_to_display(gpcr)   
    affected_vars=[]
    unaffected_vars=[]
    unknown_vars=[]
    rep_chains=set()
    for var in gpcr_vars:
        affected_pred=var["affected_pred"]
        if affected_pred:
            if affected_pred=="?":
                unknown_vars.append("(resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
            else:
                affected_vars.append("(resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                #print("%s\t%s" % (var["gpcr_num"],", ".join(var["affected_items_pred"])))
        else:
            unaffected_vars.append("(resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
        rep_chains.add("chain %s" % var["pdb_pos"]["chain"])

    unknown_vars_s=" or ".join(unknown_vars)
    affected_vars_s=" or ".join(affected_vars)
    unaffected_vars_s=" or ".join(unaffected_vars)

    gpcr_sel="protein and (%s)" % " or ".join(rep_chains)

    #Create htmd molecule of pdb
    #mol = Molecule("/home/mariona/Documents/PhD/Discoverx/mapping/pdb_files/"+pdb+".pdb")
    mol = Molecule("/home/martalo/Documentos/TFM/GPCR_variants/Data/pdb_files/"+pdb+".pdb")
    #mol.reps.remove()
    mol.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon')
    if unknown_vars_s:
        mol.reps.add(sel=unknown_vars_s ,color="#ffdd00",style='licorice')#yellow
    if affected_vars_s:
        mol.reps.add(sel=affected_vars_s ,color="#f9020a",style='licorice')#red
    if unaffected_vars_s:
        mol.reps.add(sel=unaffected_vars_s ,color="#00c631",style='licorice')#green
    mol.center()
    #mol.view()
    return mol
#{'interact', 'ptm', 'arrestinInt', 'sodiumP', 'microSwitch', 'DisGeNet_disease', 'activation', 'gprotInt', 'sift_poloP'}

In [5]:
def create_reps_affected(gpcr,var_info,show_aff_items=False,restrict_vars=False):
    plot_only_del=True
    affected_items_d={'interact': {"color":"#3875B1","varsel":[]}, #dark blue
                      'ptm': {"color":"#F67D26","varsel":[]}, #orange
                      'arrestinInt+gprotInt': {"color":"#3EA233","varsel":[]}, #green 
                      'sodiumP': {"color":"#CD1E31","varsel":[]}, #red
                      'microSwitch': {"color":"#9161BB","varsel":[]}, #purple
                      'DisGeNet_disease': {"color":"#87554D","varsel":[]}, #brown
                      'activation':{"color":"#DD70C1","varsel":[]}, #pink
                      'sift_poloP':{"color":"#B8BF30","varsel":[]} #yellow-ish green
                     } #https://www.tableau.com/about/blog/2016/7/colors-upgrade-tableau-10-56782
    
    name=var_info[gpcr]["name"]
    pdb=var_info[gpcr]["pdb"]
    print(pdb)
    is_model=var_info[gpcr]["is_model"]
    gpcr_vars=[]
    if len(list(var_info[gpcr].keys())) >1:
        gpcr_vars=var_to_display(gpcr)   
    affected_vars=[]
    unaffected_vars=[]
    unknown_vars=[]
    rep_chains=set()
    for var in gpcr_vars:
        if  restrict_vars and var["gpcr_num"] not in restrict_vars:
            continue
        affected_pred=var["affected_pred"]
        if affected_pred:
            if affected_pred=="?":
                unknown_vars.append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
            else:
                affected_vars.append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                if show_aff_items:
                    print("%s\t%s" % (var["gpcr_num"],", ".join(var["affected_items_pred"])))
                affected_items=var["affected_items_pred"] 
                if "sodiumP" in affected_items:
                    affected_items_d["sodiumP"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "microSwitch" in affected_items:
                    affected_items_d["microSwitch"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "activation" in affected_items:
                    affected_items_d["activation"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "ptm" in affected_items:
                    affected_items_d["ptm"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "arrestinInt" in affected_items or "gprotInt" in affected_items:
                    affected_items_d["arrestinInt+gprotInt"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "interact" in affected_items:
                    affected_items_d["interact"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "DisGeNet_disease" in affected_items:
                    affected_items_d["DisGeNet_disease"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                if "sift_poloP" in affected_items:
                    affected_items_d["sift_poloP"]["varsel"].append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
                    continue
                    
        else:
            unaffected_vars.append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
        rep_chains.add("chain %s" % var["pdb_pos"]["chain"])

    unknown_vars_s=" or ".join(unknown_vars)
    affected_vars_s=" or ".join(affected_vars)
    unaffected_vars_s=" or ".join(unaffected_vars)
    gpcr_sel="protein and (%s)" % " or ".join(rep_chains)

    #Create htmd molecule of pdb
    mol = Molecule("/home/martalo/Documentos/TFM/GPCR_variants/Data/pdb_files/"+pdb+".pdb")
    #mol.reps.remove()
    mol.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon')
    if unaffected_vars_s:
        mol.reps.add(sel=unaffected_vars_s ,color="#f4f8ff",style='VDW')#green
    for aff_item in affected_items_d.keys():
        if affected_items_d[aff_item]["varsel"]:
            mysel=" or ".join(affected_items_d[aff_item]["varsel"])
            mol.reps.add(sel=mysel ,color=affected_items_d[aff_item]["color"],style='VDW')
    if unknown_vars_s:
        mol.reps.add(sel=unknown_vars_s ,color="#adadad",style='VDW')#grey
    mol.center()
    #mol.view()
    return mol
#{'interact', 'ptm', 'arrestinInt', 'sodiumP', 'microSwitch', 'DisGeNet_disease', 'activation', 'gprotInt', 'sift_poloP'}

In [11]:

#test
gpcr="HRH1"
(var_info,all_gpcrs)=var_pos_in_pdb()
print(len(all_gpcrs))# total GPCRs with info of their variants
# all_gpcrs list of the names of the GPCRs

# var_info: list of dictionaries for each GPCR
# as 3rd element the dictionary jey is 'vars', this key has a value that is a list
# containing the info of all the variants of each GPCR
#print(var_info[gpcr])

print(var_info[gpcr]["vars"][3])# example, first variant of HRH1, 'raw' values

#----
gpcr_vars=var_to_display(gpcr) # curated values of the variants-> if it will have effect and where
print(gpcr_vars[0])

for var in gpcr_vars:
    affected_pred=var["affected_pred"]
    if affected_pred:
        if affected_pred=="?":
            unknown_vars.append("(name CA and resid "+var["pdb_pos"]["pos"]+" and chain "+var["pdb_pos"]["chain"]+")")
        


46
{'gpcrdb': '', 'seqN': '19', 'var_aa': 'K', 'wt_aa': 'N', 'sift_poloP': False, 'DisGeNet_disease': False, 'gprotInt': False, 'arrestinInt': False, 'activation': False, 'microSwitch': False, 'sodiumP': False, 'ptm': False, 'interact': False}
HRH1 HRH1
{'affected_pred': False, 'affected_items_pred': [], 'pdb_pos': {'aa': 'V', 'pos': '31', 'chain': 'A'}, 'gpcr_num': '1x36', 'aa_type_changed': False}


A. 'var_info'
- gprotInt: T/F/? 'GProteinInteraction'-> Infered G protein interaction (col W), True if 'putative'.

- arrestinInt: T/F/? 'ArrestinInteraction'-> Infered arrestin interaction (col X), True if 'putative'.

- sodiumP: T/F/? 'SodiumPocket'-> SODIUM allosteric sites PMIDid: 24767681 (col AA) True if 'yes'.

- interact: T/F/? 'LB_structure'-> ligand binding interaction observed in a crystal structure (col AH), True if 'interacting'.

- microSwitch: T/F/? 'MicroSwitch'-> Microswitch positions including the PIF motif PMIDid: 22300046 (col Z) True if 'yes'.

- DisGeNet_disease: T/F/? 'DisGeNet disease Name' -> DisGeNet disease Name (col AE) 

- activation: T/F/? 'ActivationPathway'-> Class A activation pathway positions: '3x46','6x37','7x53', PMIDid: 27525504 (col Y), True if 'active'.

- sift_poloP: T/F if 'sift_word'-> sorting intolerant from tolerant, category (col S) or 'polyphen_word'-> polymorphism phenotypingt, category (col U) are 'deleterious'/'damaging' -> True or 'tolerated'/'benign' -> False

- ptm: T/F/?, 'PTMsite'-> PTM site (yes/no) (col AG), True if 'yes'.

B. 'gpcr_vars':
- affected_pred: if some of the above is True
- affected_items_pred: which of the above are True
- pdb_pos: of the variant
- gpcr_num: of the variant
- aa_type_changed: to know if homologous/non-homologous


In [7]:
# The following file 'var_info.json' comes from the code in 'Variants_general/create_csv.py':
##(var_info,all_gpcrs)=var_pos_in_pdb()
##out_name="var_info.json"
##with open("//home/martalo/Documentos/TFM/GPCR_variants/Results/gpcr_to_pdb/"+out_name, 'w') as outfile:
##    json.dump(var_info, outfile)
# So it stores the variants info for the GPCRs

json_file=open("/home/martalo/Documentos/TFM/GPCR_variants/Results/gpcr_to_pdb/var_info.json")
json_str = json_file.read()
var_info=pd.io.json.loads(json_str)
#for gpcr in var_info.keys():
#    if len(list(var_info[gpcr].keys())) >1:
#        var_to_display(gpcr)    
all_gpcrs=list(var_info.keys())


In [8]:
create_reps_affected("HRH1",var_info).view()

3RZE
HRH1 HRH1


2020-10-02 10:11:39,521 - moleculekit.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2020-10-02 10:11:39,525 - moleculekit.molecule - INFO - Removed 21 atoms. 3534 atoms remaining in the molecule.


_ColormakerRegistry()

NGLWidget()

Display an example of a GPCR with the computed variants (in the file myprot_list_strucNvar.csv) in different colors depending on the 'True' values of 'var_info'. 

In [9]:
create_reps_affected("DRD2",var_info).view()

6CM4
DRD2 DRD2


NGLWidget()

In [ ]:
#create_reps_affected("driv",var_info).view()


In [16]:
create_reps_affected("CHRM1",var_info).view()


5CXV
CHRM1 CHRM1


NGLWidget()

In [17]:
create_reps_affected("HTR1B",var_info).view()


4IAQ
HTR1B HTR1B


NGLWidget()

In [19]:
gpcr=all_gpcrs.pop()
create_reps_affected("CALCR",var_info).view()

5UZ7
CALCR CALCR


FileNotFoundError: File /home/martalo/Documentos/TFM/GPCR_variants/Data/pdb_files/5UZ7.pdb was not found.

In [39]:
gpcr=all_gpcrs.pop()
create_reps_affected("FFAR1",var_info).view()


5TZY
FFAR1


FileNotFoundError: [Errno 2] No such file or directory: '/home/mariona/Documents/PhD/Research_projects/Pharmacogenomics/Results/gpcr_to_pdb/5TZY_conv.json'

In [15]:
gpcr=all_gpcrs.pop()
create_reps_affected("FSHR",var_info).view()

3VG9


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [16]:
gpcr=all_gpcrs.pop()
create_reps_affected("ADRB2",var_info).view()

3SN6


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [17]:
gpcr=all_gpcrs.pop()
create_reps_affected("CXCR4",var_info).view()

3ODU


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()
2018-11-12 17:17:57,140 - htmd.molecule.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2018-11-12 17:17:57,144 - htmd.mole

NGLWidget()

In [18]:
gpcr=all_gpcrs.pop()
create_reps_affected("CCR8",var_info).view()

5UIW


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()
2018-11-12 17:17:58,842 - htmd.molecule.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2018-11-12 17:17:58,845 - htmd.mole

NGLWidget()

In [19]:
gpcr=all_gpcrs.pop()
create_reps_affected("MCHR1",var_info).view()

6B73


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [20]:
gpcr=all_gpcrs.pop()
create_reps_affected("CNR2",var_info).view()

5TGZ


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()
2018-11-12 17:18:00,984 - htmd.molecule.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2018-11-12 17:18:00,987 - htmd.mole

NGLWidget()

In [21]:
gpcr=all_gpcrs.pop()
create_reps_affected("GPR18",var_info).view()

6B73


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [25]:
gpcr=all_gpcrs.pop()
create_reps_affected("TAAR1",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [26]:
gpcr=all_gpcrs.pop()
create_reps_affected("HRH4",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [27]:
gpcr=all_gpcrs.pop()
create_reps_affected("HRH3",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()
2018-11-07 17:18:21,685 - htmd.molecule.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2018-11-07 17:18:21,689 - htmd.mole

NGLWidget()

In [28]:
gpcr=all_gpcrs.pop()
create_reps_affected("HRH2",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [29]:
gpcr=all_gpcrs.pop()
create_reps_affected("HRH1",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()
2018-11-07 17:18:24,004 - htmd.molecule.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2018-11-07 17:18:24,007 - htmd.mole

NGLWidget()

In [30]:
gpcr=all_gpcrs.pop()
create_reps_affected("DRD5",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [31]:
gpcr=all_gpcrs.pop()
create_reps_affected("DRD4",var_info).view()

/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()
2018-11-07 17:18:26,546 - htmd.molecule.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2018-11-07 17:18:26,550 - htmd.mole

NGLWidget()

In [93]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

dopamine receptor D3 (DRD3)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [94]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

dopamine receptor D2 (DRD2)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [95]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

dopamine receptor D1 (DRD1)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [96]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

adrenoceptor beta 3 (ADRB3)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [97]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

adrenoceptor beta 2 (ADRB2)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [98]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

adrenoceptor beta 1 (ADRB1)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [99]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

adrenoceptor alpha 2C (ADRA2C)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [100]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

adrenoceptor alpha 2B (ADRA2B)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()

In [101]:
gpcr=all_gpcrs.pop()
create_reps_affected(gpcr,var_info).view()

adrenoceptor alpha 2A (ADRA2A)


/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:599: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['rotations'] = parsedsymmetry[['rot1', 'rot2', 'rot3']].as_matrix().reshape((numcopies, 3, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  crystalinfo['translations'] = parsedsymmetry['trans'].as_matrix().reshape((numcopies, 3))
/home/mariona/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:606: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  serials = parsedtopo.serial.as_matrix()


NGLWidget()